In [2]:

from qiskit import QuantumCircuit, transpile
from qiskit.tools.monitor import job_monitor
from azure.quantum.qiskit import AzureQuantumProvider
import random

import re
import numpy as np





In [3]:
provider = AzureQuantumProvider (
    resource_id = "/subscriptions/b1d7f7f8-743f-458e-b3a0-3e09734d716d/resourceGroups/aq-hackathons/providers/Microsoft.Quantum/Workspaces/aq-hackathon-01",
    location = "eastus")

qpu_backend = provider.get_backend("ionq.qpu")


def QRNG(ngates):
    """
    Given the number of gates in the deck, returns a quantum random index
    """
    nqubits = 6
    circ = QuantumCircuit(nqubits,nqubits)
    circ.name = f'{nqubits}-qubit QRNG'
    for reg in range(nqubits):
        circ.h(reg)             # putting it into a superposition
    circ.measure([reg for reg in range(nqubits)],[reg for reg in range(nqubits)])
    
    qpu_backend = provider.get_backend("ionq.qpu")
    qpu_job = qpu_backend.run(circ, shots=1)    # just one shot to select from the set of possible bitstrings
    result = qpu_job.result()
    
    bitstring = list(result.get_counts(circ).keys())[0] # extracts the bistring that was measured
    randnum = int(bitstring,2)
    gateindex = int(randnum/(2**nqubits-1) * (ngates-1)) # rescales the qubit's domain to the domain we need from the gate deck
    
    return gateindex

In [28]:

class QuantumPoker:
    def deal(self,hand):
        """Tops off the players hand to ensure they have three cards"""
        while len(hand)<3:
            
            if len(self.deck)==0:
                return
            # here we choose whether we want to draw the cards via quantum or classical randomness
            if self.sim:
                hand.append(self.deck.pop(np.random.randint(len(self.deck))))
            else:
                hand.append(self.deck.pop(QRNG(len(self.deck))))

              

    def __init__(self,num_players,sim=True):
        """Initializes a game with the amount of players specified"""
        self.sim=sim

        
        self.deck=[]
        
        # fills the deck with the given ratios of gates
        for i in [("x",3),("z",1),("y",2),("h",2),("c",2),("t",1),("s",1)]:
            for j in range(i[1]*num_players):
                
                self.deck.append(i[0])
        
        self.game={}
        for i in range(num_players):
            # creates a 6-qubit empy circuit for each player
            name="player"+str(i)
            cards=[]
            self.deal(cards)
            self.game[name]={"hand":QuantumCircuit(6,6),"cards":cards}
        
        self.goal=list(np.random.randint(2,size=6)) # arbitrary objective bitstring, e.g. "100101"
        print("Your goal is "+ "".join(map(str,self.goal)))
        

    

    def turn(self,player,moves):
        """This does a players turn based on the cards
            takes a player object and a tuple of form (<gate>, (<person target>, <qubit target>[, <control qubit>]))"""
        # moves is a list of move entries, with each move being in the format:
        # (<gate>, (<person target>, <qubit target>[, <control qubit>]))
        for move in moves:
            target="player"+str(move[1][0])
            bit=int(move[1][1])
            
            
            # these apply the chosen gate
            if move[0] in player["cards"]:
                player["cards"].remove(move[0])
            if move[0]=="x":
                self.game[target]["hand"].x(bit)
                
            elif move[0]=="y":
                self.game[target]["hand"].y(bit)
            elif move[0]=="z":
                self.game[target]["hand"].z(bit)
            elif move[0]=="c":
                self.game[target]["hand"].c(move[1][2],bit)
            elif move[0]=="h":
               self.game[target]["hand"].h(bit)
            elif move[0]=="t":
                self.game[target]["hand"].t(bit)
            elif move[0]=="s":
                self.game[target]["hand"].s(bit)

        self.deal(player["cards"])



    def sorter(e):
        """Sorts by second element"""
        return e[1]
        
    def begin(self):
        last=False
        while(True):
            if len(self.deck)==0:
                print("Last Turn")
                last=True

                
        
            for playerd in self.game:

                #introduces the turn
                player=self.game[playerd]
                print("It is "+playerd+"'s turn")
                if len(player["cards"])>0:
                    print("Your cards are :"+str(player["cards"]))
                    print("Here all the circuits")
                    for show in self.game:
                        circ=self.game[show]["hand"]
                        print(circ.draw("text"))
                else:
                    print("You have no cards")

                num = input("How many cards would you like to play?")
                if not num: continue # passes
                
                moves=[]
                # print(player["hand"])
                if num>len(player["cards"]):
                    print("Trying to cheat are we?")

                for i in range(min(int(num),len(player["cards"]))):

                    #adds a move to the moves list for each gate used
                    gate= input("Which gate shall you use?")
                    person= input("Who will be your target?")
                    bit= input("Which bit?")
                    
                    if gate=="c":
                        cbit=input("Which one is the control bit?")
                        target=(person,bit,cbit)
                        moves.append((gate,target))
                    else:
                        target=(person,bit)
                        moves.append((gate,target))
                
                self.turn(player,moves) # applies the gates to the designated circuit(s)
            if last:
                break
        
        scores=[]
        for player in self.game:
            
            # measures the player's circuit once and records the observed bitstring
            circ=self.game[player]["hand"]
            circ.measure([i for i in range(6)],[i for i in range(6)])

            
            player_job=qpu_backend.run(circ,shots=1)
            result=player_job.result()
            bitstring= list(result.get_counts(circ).keys())[0]
            
            # compares the observed bitstring to the goal bitstring by summing the number of indices they differ
            tot=0
            for i in str(bin((int("".join(map(str,self.goal)),2)^int(bitstring,2)))):
                if i in ("1","0"):
                    tot+=1
            scores.append((player,tot))
        scores.sort(key=sorter(),reverse=True)
        winners=[]
        top=scores[0][1]
        for i in scores:
            if i[1]==top:
                winners.append(i[0])
            else:
                break
        print("Game over!")
        print("The top scorers were: "+ winners)